In [ ]:
    import ast, abc, jinja2, sys, nbconvert, nbformat, contextlib, emoji, collections, IPython, mistune, doctest, re, traitlets, textwrap, types, typing as t, importlib, importnb,  jinja2, builtins, jsonschema
    with __import__('importnb').Notebook():
        try:
            from . import tangle
        except:
            from pidgin import tangle, specification
            
    with tangle.Pidgin(): 
        try:
            from . import shell, formatter
        except:
            from pidgin import shell, formatter
    try:
        from . import specification
    except:
        from pidgin import specification


In [ ]:
    import string

In [ ]:
    class BaseFormatter(shell.Shell, IPython.core.formatters.BaseFormatter):
        special_formatters = traitlets.Tuple()                   
        def init_display_formatter(Formatter):
            IPython.core.formatters.BaseFormatter.__init__(Formatter, **Formatter.parent.display_formatter._trait_values)
            for object in  Formatter.special_formatters: 
                if len(object) == 2:
                    Formatter.for_type(*object)
                if len(object) == 2:
                    Formatter.for_type_by_name(*object)

        def run_cell(Formatter, text, *args, **kwargs): 
            object = Formatter.format(text)
            if Formatter.publish(text):
                Formatter.parent.display_pub.publish(*object, transient={'raw': True})
                
        def publish(Formatter, str) -> bool: return True

In [ ]:
    class Formatter(BaseFormatter, IPython.core.formatters.DisplayFormatter):
        def init_display_formatter(Formatter):
            IPython.core.formatters.DisplayFormatter.__init__(Formatter, **Formatter.parent.display_formatter._trait_values)
            for object in  Formatter.special_formatters: 
                if len(object) == 2:
                    Formatter.mimebundle_formatter.for_type(*object)
                if len(object) == 2:
                    Formatter.mimebundle_formatter.for_type_by_name(*object)

            Formatter.display_formatter = Formatter


        def format(Formatter, object, include=None, exclude=None):
            if isinstance(object, str):
                object = Formatter.render(object)
            if isinstance(object, tuple) and len(object)==2:
                data, metadata = object
            else:
                data, metadata = super().format(object, include=include, exclude=exclude)
            return data, metadata
        
        def render(self, text): return text
        
        def publish(Formatter, str) -> bool:
            return bool(str.partition('\n')[0].strip()) and not str.startswith('//')            

In [ ]:
    class Dollar(Formatter):
        def render(Dollar, text):
            return string.Template(text).safe_substitute(**{**vars(builtins), **Dollar.user_ns})

In [ ]:
    if __name__ == '__main__':
        %reload_ext pidgin